In [2]:
import merkletools
from web3 import Web3
from solcx import compile_files
import client.utils as utils
import client.ram as ram
import copy

In [2]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))

In [19]:
contract_id, compiled_sol = compile_files(["missmatch.sol"], output_values=['abi', 'bin']).popitem()

w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))

missmatch = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

tx_hash = missmatch.constructor().transact({'from': w3.eth.accounts[1]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

Missmatch = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])


In [2]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
contract_id, compiled_sol = compile_files(["missmatch.sol"], output_values=['abi', 'bin']).popitem()
Missmatch = w3.eth.contract(address='0x6c52d2673E265983bC6F10C6DC8a3D3A3d7d1b1B', abi=compiled_sol['abi'])

In [3]:
code = ram.RamCoder(10)
code.load(5, 0)
code.load(7, 1)
code.load(1, 5)
code.add(0, 2, 2)
code.sub(1, 5, 1)
code.jnz(1, 2)
code.end()

ram1 = ram.Ram(code.source)
ram1()

mt1 = utils.reduced_config_mt(ram1.rc_list)

corrupted_rc_list = copy.deepcopy(ram1.rc_list)
corrupted_rc_list[12]["args"] = [1, 2, 3]
corrupted_rc_list[13]["args"] = [1, 2, 3]
corrupted_rc_list[14]["args"] = [1, 2, 3]
corrupted_rc_list[15]["args"] = [1, 2, 3]
corrupted_rc_list[16]["args"] = [1, 2, 3]
corrupted_rc_list[17]["args"] = [1, 2, 3]
corrupted_rc_list[18]["args"] = [1, 2, 3]

mt2 = utils.reduced_config_mt(corrupted_rc_list)

'62b40c8fabfee5a07bd5bc866a0c48ec74ea57d293ab40be0e156da9d07f4658'

In [14]:
ram1.rc_list

[{'PC': 0,
  'OpCode': <OpCode.LOAD: 1>,
  'args': [5, 0],
  'args_values': [0],
  'args_proofs': [{'root': '0x8a476f7ecd3d62b0dbb7badd58464f369116dc3258340657ef6ac50ae5842e35',
    'leaf': '0x011b4d03dd8c01f1049143cf9c4c817e4b167f1d1b83e5c6f0f10d89ba1e7bce',
    'arr': ['0xf9cffb62426e025d905b05f86e9d0dabea946af87ad8e66fadab82c99bc568eb',
     '0x789833d9d7f0fe00d382584b02a9ff3bc6b4ff6142c11b2e060f48f8bd34a85b',
     '0x5d44597fe4efa5395c34e82446e0884c0a438998e9b900b229849130774ad30c',
     '0xc9e5765a3586e722bff413c64fd4af2894367e5ed54bd154799a97050da339d4'],
    'pos': [True, True, True, True]}],
  'store_mt_root': '0x8a476f7ecd3d62b0dbb7badd58464f369116dc3258340657ef6ac50ae5842e35',
  'next_op_proof': {'root': '0x99d952b217b7609822d97e5d647e949d74dbe23cd963eafe2ada4b08bdf00bf5',
   'leaf': '0xa4c498d414c081df92f422063323d058d2fa3614c0d0f9c666064602edf827d6',
   'arr': ['0xc32baddb267bde6f632f084dc431e8ec2f41f9ae3be9cab264f12de160fec9fe',
    '0xac9f8622137f68b546293d578e30bdabfa320

In [4]:
tx_hash = Missmatch.functions.start_committed_bs(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    '0x' + mt2.get_merkle_root(),
    w3.eth.accounts[1],
    w3.eth.accounts[2],
    '0x' + ram1.code_mt.get_merkle_root()).transact({'from': w3.eth.accounts[1]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [39]:
import asyncio
mt = [mt1, mt2]

def handle_event(event, conractor):
    middle = event['args']['middle']
    Missmatch.functions.send_rc_proof(utils.translate_proof(mt[conractor-1], middle)).transact({'from': w3.eth.accounts[conractor]})
    print(f'conractor = {conractor} middle = {middle}')

def handle_verify_committed_step(event, rc_list, conractor):
    low = event['args']['low']
    rc = rc_list[low]
    return Missmatch.functions.send_committed_step(
        PC = rc['PC'], 
        OpCode = int(rc['OpCode']), 
        args = rc['args'], 
        args_values = utils.sterilize_ints(rc['args_values']), 
        args_proof = rc['args_proofs'],
        store_mt_root = rc['store_mt_root'],
        nextOpCode = int(rc_list[low+1]['OpCode']),
        nextArgs = utils.sterilize_ints(rc_list[low+1]['args']),
        next_op_proof = rc['next_op_proof']
        ).transact({'from': w3.eth.accounts[conractor]})
    

async def log_loop_c1(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 1)
        for event in event_filter_verify.get_new_entries():
            pass
            #handle_verify_committed_step(event, ram1.rc_list, 1)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(event_filter, event_filter_verify, poll_interval):
    for x in range(8):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 2)
        await asyncio.sleep(poll_interval)

async def main():
    event_filter_c1 = Missmatch.events.request_proof_of_middle.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_c2 = Missmatch.events.request_proof_of_middle.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_verift_c1 = Missmatch.events.request_proof_of_committed_step.createFilter(fromBlock=1 ,toBlock='latest')
    event_filter_verift_c2 = Missmatch.events.request_proof_of_committed_step.createFilter(fromBlock=1 ,toBlock='latest')

    await asyncio.gather(
        log_loop_c1(event_filter_c1, event_filter_verift_c1, 1),
        log_loop_c2(event_filter_c2, event_filter_verift_c2, 1))


await main()


In [5]:
import asyncio
mt = [mt1, mt2]

def handle_event(event, conractor):
    middle = event['args']['middle']
    Missmatch.functions.send_rc_proof(utils.translate_proof(mt[conractor-1], middle)).transact({'from': w3.eth.accounts[conractor]})
    print(f'conractor = {conractor} middle = {middle}')

def handle_verify_committed_step(event, rc_list, conractor):
    low = event['args']['low']
    rc = rc_list[low]
    return Missmatch.functions.send_committed_step(
        PC = rc['PC'], 
        OpCode = int(rc['OpCode']), 
        args = rc['args'], 
        args_values = utils.sterilize_ints(rc['args_values']), 
        args_proof = rc['args_proofs'],
        store_mt_root = rc['store_mt_root'],
        nextOpCode = int(rc_list[low+1]['OpCode']),
        nextArgs = utils.sterilize_ints(rc_list[low+1]['args']),
        next_op_proof = rc['next_op_proof']
        ).transact({'from': w3.eth.accounts[conractor]})
    

async def log_loop_c1(poll_interval):
    old = []
    for x in range(8):
        dif = list(set(Missmatch.events.request_proof_of_middle.getLogs(fromBlock=1)) - set(old))
        old = list(set(Missmatch.events.request_proof_of_middle.getLogs(fromBlock=1)))
        for PairCreated in dif:
            handle_event(PairCreated, 1)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(poll_interval):
    old = []
    for x in range(8):
        dif = list(set(Missmatch.events.request_proof_of_middle.getLogs(fromBlock=1)) - set(old))
        old = list(set(Missmatch.events.request_proof_of_middle.getLogs(fromBlock=1)))
        for PairCreated in dif:
            handle_event(PairCreated, 2)
        await asyncio.sleep(poll_interval)

async def main():
    await asyncio.gather(
        log_loop_c1(1),
        log_loop_c2(1))


await main()

conractor = 1 middle = 15
conractor = 2 middle = 15
conractor = 1 middle = 7
conractor = 2 middle = 7
conractor = 1 middle = 11
conractor = 2 middle = 11
conractor = 1 middle = 13
conractor = 2 middle = 13
conractor = 1 middle = 12
conractor = 2 middle = 12


In [6]:

handle_verify_committed_step(Missmatch.events.request_proof_of_committed_step.getLogs(fromBlock=1)[0], ram1.rc_list, 1)



HexBytes('0x777963dc16168e4b552c43e133aa413ea29e5bda0e963ad988ce7d89fa74a1c9')

In [7]:
Missmatch.events.request_proof_of_middle.getLogs(fromBlock=1)
Missmatch.events.request_proof_of_committed_step.getLogs(fromBlock=1)
Missmatch.functions.winner().call()


'0xa86E7046b55A160E85A38A50e95690Db38523c1D'

In [7]:
ram1.rc_list[13]['next_op_proof']

{'root': '0x99d952b217b7609822d97e5d647e949d74dbe23cd963eafe2ada4b08bdf00bf5',
 'leaf': '0x48fbabcb9ec6aa504d8f02bdae5d8f67588b81437148ab370766a439b97d66fc',
 'arr': ['0xcda6debc3b2fa97037fa318ad503c246d53dc513443d6cbccf9f5570e5e7ce0e',
  '0xe882cf2608db76ec2c0b941723b8305a70771a2b74030b800da37a0d0702be5a',
  '0xe0fb745085fa683a38b7c59e5dcd8e3040349255a3051326f4f0b5789a463a45'],
 'pos': [True, False, True]}